In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

2024-08-05 13:05:52.026313: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 13:05:52.056817: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 13:05:52.610264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Carrega a base de dados a partir de seu caminho
data = np.load('dataset.npz', allow_pickle=True)
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']

In [3]:
# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train_categorical = to_categorical(y_train, num_classes)

In [4]:
num_classes

7

In [5]:
# Scale
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [6]:
#Model
initializer = keras.initializers.GlorotNormal(seed=12227)

inputs_layer = keras.layers.Input(shape=(X_test_sc.shape[1],))

H = keras.layers.Dense(units=64, activation='relu')(inputs_layer)
shortcut=H
H = keras.layers.Dense(units=128, activation='relu')(H)
H = keras.layers.Dense(units=256, activation='relu')(H)
H = keras.layers.Dense(units=512, activation='relu')(H)
H = keras.layers.Dense(units=1024, activation='relu')(H)
H = keras.layers.Dense(units=64, activation='relu')(H)
H=H+shortcut
outputs_layer = keras.layers.Dense(units=num_classes, activation='softmax')(H)

model = keras.models.Model(inputs=inputs_layer, outputs=outputs_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2024-08-05 13:05:53.092933: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: UNKNOWN ERROR (100)


In [7]:
def lr_schedule(epoch, lr):
    if epoch < 50:
        return 0.001
    elif epoch < 100:
        return 0.0001
    elif epoch < 150:
        return 0.00001
    else:
        return 0.00001/epoch
    
scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)

history = model.fit(X_train_sc, y_train_categorical, epochs=200, batch_size=64, callbacks=[scheduler])

Epoch 1/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4210 - loss: 1.5393 - learning_rate: 0.0010
Epoch 2/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6298 - loss: 1.0219 - learning_rate: 0.0010
Epoch 3/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6703 - loss: 0.8801 - learning_rate: 0.0010
Epoch 4/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7127 - loss: 0.7752 - learning_rate: 0.0010
Epoch 5/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7456 - loss: 0.7078 - learning_rate: 0.0010
Epoch 6/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7934 - loss: 0.5883 - learning_rate: 0.0010
Epoch 7/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7969 - loss: 0.5463 - learning_rate: 0.0010
Epoch 8/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8121 - loss: 0.5013 - learning_rate: 0.0010
Epoch 9/200
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8297 - loss: 0.4694 - learning_rate: 0.0010
Epoch 10/200
53/53 

In [8]:
y_pred_prob = model.predict(X_test_sc)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [9]:
y_pred_classes = np.argmax(y_pred_prob, axis=1)
# Gera um DataFrame no formato esperado da submissão
num_samples = X_test.shape[0]
submission_df = pd.DataFrame({
    'ID': np.arange(1, num_samples + 1),
    'Prediction': y_pred_classes
})

# Salva o arquivo CSV no diretório atual
submission_df.to_csv('submission_task4_v3.csv', index=False)